In [ ]:
# inference.ipynb

import pandas as pd
import joblib
import os
import numpy as np
from IPython.display import display

# 1. 모델 및 인코더 로드
try:
    model = joblib.load('pitcher_model.pkl')
    le = joblib.load('pitch_type_encoder.pkl')
    print("모델 및 인코더 로드 성공! 추론 준비 완료.")
except FileNotFoundError:
    print("오류: 모델 파일이나 인코더 파일을 찾을 수 없습니다. training.ipynb를 먼저 실행했는지 확인하세요.")
    exit()

# 2. 추론에 사용할 특징량 정의
FEATURES = ['balls', 'strikes', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'inning']
PITCHER_NAME = "Logan Allen" # 추론 대상 투수 이름


# 3. 추론 함수 정의
def predict_pitch(balls, strikes, on_1b, on_2b, on_3b, outs_when_up, inning):
    """
    주어진 상황을 바탕으로 다음 투구 구종의 확률을 예측하고 가장 높은 구종을 반환합니다.
    (주의: on_1b, on_2b, on_3b는 주자 유무: 1=있음, 0=없음)
    """
    
    # 입력 데이터를 DataFrame 형태로 변환 
    input_data = pd.DataFrame({
        'balls': [balls],
        'strikes': [strikes],
        'on_1b': [on_1b],
        'on_2b': [on_2b],
        'on_3b': [on_3b],
        'outs_when_up': [outs_when_up],
        'inning': [inning]
    })
    
    # 예측 확률 계산
    proba = model.predict_proba(input_data)[0]
    
    # 예측된 인코딩된 라벨 (가장 높은 확률)
    predicted_encoded = model.predict(input_data)[0]
    
    # 인코딩된 라벨을 실제 구종 이름으로 디코딩
    predicted_pitch = le.inverse_transform([predicted_encoded])[0]
    
    # 확률표 만들기
    proba_df = pd.DataFrame({
        'Pitch Type': le.classes_,
        'Probability': proba
    }).sort_values(by='Probability', ascending=False).reset_index(drop=True)
    
    
    print(f"\n[투수: {PITCHER_NAME}] 현재 상황: {balls}B-{strikes}S, {outs_when_up}아웃, {inning}이닝")
    print("-" * 40)
    print(f" 최종 예측 구종: {predicted_pitch}")
    print("-" * 40)
    display(proba_df)


# 4. 추론 예시 (총 8개 시나리오)

print("========== 다음 투구 예측 시뮬레이션 시작 ========== \n")

# --- A. 결정구 상황 (0-2) ---
# 예시 1: 유리한 카운트 (결정구, 주자 없음)
# 0-2 카운트, 주자 없음, 5회
predict_pitch(balls=0, strikes=2, on_1b=0, on_2b=0, on_3b=0, outs_when_up=1, inning=5)

# 예시 2: 유리한 카운트 (결정구, 2루 주자 존재)
# 0-2 카운트, 2루 주자 존재, 7회
# (주자가 있을 경우 폭투 위험이 적은 구종 선택 가능성 확인)
predict_pitch(balls=0, strikes=2, on_1b=0, on_2b=1, on_3b=0, outs_when_up=0, inning=7)


# --- B. 풀카운트 상황 (3-2) ---
# 예시 3: 풀카운트 (주자 없음, 삼진 유도 목적)
# 3-2 카운트, 주자 없음, 3회
# (삼진 유도를 위한 변화구 vs 힘으로 누르는 직구 패턴 확인)
predict_pitch(balls=3, strikes=2, on_1b=0, on_2b=0, on_3b=0, outs_when_up=2, inning=3)

# 예시 4: 풀카운트 (만루 위기, 승부 상황)
# 3-2 카운트, 만루, 9회 말 (가장 부담되는 상황)
predict_pitch(balls=3, strikes=2, on_1b=1, on_2b=1, on_3b=1, outs_when_up=2, inning=9)


# --- C. 불리한 카운트 (3-1) ---
# 예시 5: 불리한 카운트 (도망가는 상황)
# 3-1 카운트, 주자 1루, 7회
# (스트라이크를 잡기 위해 직구 계열 비율이 높아지는지 확인)
predict_pitch(balls=3, strikes=1, on_1b=1, on_2b=0, on_3b=0, outs_when_up=0, inning=7)


# --- D. 공격적인 카운트 (1-0, 2-0 등) ---
# 예시 6: 초반 공격 (공격적 카운트)
# 1-0 카운트, 주자 없음, 1회 초
# (타자가 노리고 들어올 수 있으므로 직구 계열 비율 확인)
predict_pitch(balls=1, strikes=0, on_1b=0, on_2b=0, on_3b=0, outs_when_up=0, inning=1)


# --- E. 득점권 상황 ---
# 예시 7: 득점권 (주자 2, 3루)
# 1-2 카운트, 2, 3루 주자, 6회
# (실투를 줄이고 땅볼 유도에 유리한 구종 선택 가능성 확인)
predict_pitch(balls=1, strikes=2, on_1b=0, on_2b=1, on_3b=1, outs_when_up=1, inning=6)


# --- F. 초구 상황 ---
# 예시 8: 초구 (스트라이크 잡기)
# 0-0 카운트, 주자 없음, 1회
predict_pitch(balls=0, strikes=0, on_1b=0, on_2b=0, on_3b=0, outs_when_up=0, inning=1)


print("\n========== 시뮬레이션 완료 ==========")